In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pickle

In [3]:
with open('Paruosti_duomenys.pkl', 'rb') as file:
    df_Elektra = pickle.load(file)
df_Elektra.reset_index(drop=True, inplace=True)

df_Elektra.columns

Index(['Dieninė dedamoji', 'Naktinė dedamoji', 'Pastovioji dedamoji. EUR/mėn.',
       'Viena laiko zona', 'Nord pool kaina', 'Metinė infliacija',
       'Vidutinė Temperatūra C', 'Pagaminta elektra GWh'],
      dtype='object')

In [4]:
X = df_Elektra[['Dieninė dedamoji', 'Naktinė dedamoji', 'Pastovioji dedamoji. EUR/mėn.',
                'Nord pool kaina', 'Metinė infliacija', 'Vidutinė Temperatūra C', 'Pagaminta elektra GWh']]

y = df_Elektra['Viena laiko zona']

In [5]:
y

0      0.105
1      0.105
2      0.105
3      0.105
4      0.105
       ...  
110    0.203
111    0.198
112    0.198
113    0.198
114    0.181
Name: Viena laiko zona, Length: 115, dtype: float64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
reg = LinearRegression().fit(X_train, y_train)

In [10]:
pred = reg.predict(X_test)

In [11]:
pred

array([0.14086295, 0.10398911, 0.11314054, 0.15334013, 0.10367679,
       0.11225657, 0.1538319 , 0.15485923, 0.11436481, 0.10525641,
       0.28795114, 0.11310536, 0.15372587, 0.20438471, 0.13013063,
       0.10923648, 0.14012461, 0.11596627, 0.25055444, 0.11295516,
       0.15890217, 0.20343207, 0.10603935, 0.11992208, 0.15327466,
       0.15697351, 0.21774772, 0.12536524, 0.11026381, 0.21596271,
       0.14537816, 0.10467837, 0.12047091, 0.15599243, 0.10306467,
       0.11725856, 0.13097185, 0.15271139])

In [12]:
spejimai = pd.Series(data=pred, name='Spėjimas')

In [13]:
rezultatas = pd.concat([y_test.reset_index(), spejimai], axis=1)[['Viena laiko zona', 'Spėjimas']]

In [14]:
rezultatas.head()

,Viena laiko zona,Spėjimas
0,0.141,0.140863
1,0.105,0.103989
2,0.114,0.113141
3,0.149,0.153340
4,0.105,0.103677


In [15]:
reg.score(X_test, y_test)

0.9892734052537978

In [16]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# Normalizavimas ir polinominės funkcijos
poly = PolynomialFeatures(degree=1)
scaler = StandardScaler()
model = make_pipeline(scaler, poly, LinearRegression())

# Apmokome modelį su mokymo duomenimis
model.fit(X_train, y_train)

# Kryžminė validacija tikslumui įvertinti
scores = cross_val_score(model, X, y, cv=10, scoring='r2')
print("Kryžminės validacijos R^2 reikšmės:", scores)
print("Vidutinis R^2 tikslumas:", scores.mean())

Kryžminės validacijos R^2 reikšmės: [  0.           0.96231983  -1.61968137 -22.49506078   0.73768698
   0.8888702   -2.37445543  -0.79141642   0.91304963   0.87911582]
Vidutinis R^2 tikslumas: -2.2899571547840707


In [17]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
y_pred = cross_val_predict(model, X, y, cv=5)
scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print("Kryžminės validacijos R² reikšmės:", scores)
print("Vidutinis R² tikslumas:", scores.mean())
comparison = pd.DataFrame({'Tikrasias reikšmes': y, 'Prognozuotas reikšmes': y_pred})
print(comparison)
print("\nPirmųjų kelias reikšmės iš kryžminės validacijos padalijimų:")
for fold_idx in range(5):
    fold_start = fold_idx * len(X) // 5
    fold_end = (fold_idx + 1) * len(X) // 5
    print(f"\nPadalijimas {fold_idx + 1}:")
    print(comparison.iloc[fold_start:fold_end])

Kryžminės validacijos R² reikšmės: [0.95925235 0.17503151 0.9683229  0.29046006 0.78182094]
Vidutinis R² tikslumas: 0.6349775536175035
     Tikrasias reikšmes  Prognozuotas reikšmes
0                 0.105               0.108520
1                 0.105               0.105699
2                 0.105               0.102741
3                 0.105               0.101926
4                 0.105               0.103722
..                  ...                    ...
110               0.203               0.245757
111               0.198               0.199295
112               0.198               0.199273
113               0.198               0.199177
114               0.181               0.180039

[115 rows x 2 columns]

Pirmųjų kelias reikšmės iš kryžminės validacijos padalijimų:

Padalijimas 1:
    Tikrasias reikšmes  Prognozuotas reikšmes
0                0.105               0.108520
1                0.105               0.105699
2                0.105               0.102741
3              